## 作業

請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras [官方文件中 fit_generator 的說明](https://keras.io/models/sequential/)

In [7]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

In [10]:
batch_size = 128 
num_classes = 10 
epochs = 10
LEARNING_RATE = 1e-3
MOMENTUM = 0.95

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM),
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)       

In [12]:
from sklearn.utils import shuffle
def my_generator(x, y, batch_size):
    while True:
        for idx in range(0, len(x), batch_size): 
            batch_x, batch_y = x[idx:idx+batch_size], y[idx:idx+batch_size]
            yield batch_x, batch_y
        x, y = shuffle(x, y)

In [13]:
train_generator = my_generator(x_train, y_train, batch_size)

In [14]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=int(len(x_train)/batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
390/390 [==============================] - 72s 185ms/step - loss: 0.3830 - accuracy: 0.8967 - val_loss: 0.3130 - val_accuracy: 0.9000
Epoch 2/10
390/390 [==============================] - 81s 208ms/step - loss: 0.3116 - accuracy: 0.8998 - val_loss: 0.3044 - val_accuracy: 0.9001
Epoch 3/10
390/390 [==============================] - 89s 228ms/step - loss: 0.3034 - accuracy: 0.9001 - val_loss: 0.2953 - val_accuracy: 0.9006
Epoch 4/10
390/390 [==============================] - 95s 244ms/step - loss: 0.2969 - accuracy: 0.9004 - val_loss: 0.2931 - val_accuracy: 0.9008
Epoch 5/10
390/390 [==============================] - 98s 251ms/step - loss: 0.2920 - accuracy: 0.9007 - val_loss: 0.2865 - val_accuracy: 0.9013
Epoch 6/10
390/390 [==============================] - 99s 255ms/step - loss: 0.2874 - accuracy: 0.9011 - val_loss: 0.2782 - val_accuracy: 0.9021
Epoch 7/10
390/390 [==============================] - 100s 256ms/step - loss: 0.2823 - accuracy: 0.9014 - val_loss: 0.2759 - val_a